<a href="https://colab.research.google.com/github/bharatgirdhar/End_Assignment3/blob/main/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install torch
!pip install torchtext
!pip install spacy

In [29]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz --no-deps


     |████████████████████████████████| 12.1MB 257kB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-cp36-none-any.whl size=12047109 sha256=b29f10208b0ed22a8b66cefc3f137513ea9b22d0159701c47a62009d7ac3f3aa
  Stored in directory: /root/.cache/pip/wheels/2b/3f/41/f0b92863355c3ba34bb32b37d8a0c662959da0058202094f46
Successfully built en-core-web-sm
  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5


In [30]:
import torch
from torchtext import data

SEED = 1234
import spacy.cli 
spacy.cli.download("en")

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [31]:

from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [32]:
for i in range(len(train_data)):
  vars(train_data.examples[i]).get('text').reverse()

In [33]:

print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [34]:
print(vars(train_data.examples[0]))

{'text': ['.', 'age', 'any', 'of', 'person', 'a', 'for', 'meaningful', 'be', 'can', 'which', 'film', 'family', 'powerful', 'a', 'is', 'This', '.', 'lives', 'their', 'of', 'part', 'major', 'a', 'obviously', 'is', 'and', 'terms', 'basic', 'in', 'shown', 'is', 'family', 'the', 'of', 'faith', 'The', '.', 'sugarcoating', 'or', 'sentiment', 'undo', 'without', 'depicted', 'is', 'one', 'loved', 'a', 'of', 'illness', 'devastating', 'a', 'with', 'deal', 'family', 'his', 'and', 'Dan', 'How', '.', 'team', 'real', 'a', 'make', 'to', 'takes', 'it', 'what', 'shows', ',', 'teammates', 'his', 'and', 'coach', "'s", 'Dan', 'of', 'determination', 'and', 'commitment', 'The', '.', 'us', 'of', 'all', 'for', 'lesson', 'life', 'a', 'is', 'races', 'losing', 'and', 'winning', 'handle', 'family', 'his', 'of', 'rest', 'the', 'and', 'Dan', 'How', '.', 'throughout', 'evident', 'is', 'members', 'family', 'the', 'of', 'support', 'and', 'love', 'The', '.', 'adulthood', 'full', 'through', 'age', 'young', 'a', 'from', 'l

In [35]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [36]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [37]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

In [38]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [39]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 200865), (',', 191703), ('.', 165298), ('a', 108905), ('and', 108592), ('of', 100075), ('to', 93043), ('is', 76135), ('in', 60900), ('I', 54283), ('it', 53413), ('that', 48852), ('"', 44066), ("'s", 43361), ('this', 42376), ('-', 36805), ('/><br', 35464), ('was', 34877), ('as', 30278), ('movie', 29887)]


In [40]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f308da49158>, {'neg': 0, 'pos': 1})


In [41]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [44]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

        l = [self.embedding,self.rnn,self.fc]
        self.module_list = nn.ModuleList(l)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
               
        #embedded = [sent len, batch size, emb dim]
        text_lengths = torch.as_tensor(text_lengths.cpu(), dtype=torch.int64)
        for f in self.module_list:
          embedded = self.dropout(self.embedding(text))
        #pack sequence
          packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
          packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
          output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
          hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
          return self.fc(hidden)

In [45]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 3
BIDIRECTIONAL = True
DROPOUT = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [46]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 6,387,817 trainable parameters


In [47]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [48]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3970,  0.4024,  1.0612,  ..., -0.0136, -0.3363,  0.6442],
        [-0.5197,  1.0395,  0.2092,  ..., -0.8857, -0.2294,  0.1244],
        [ 0.0057, -0.0707, -0.0804,  ..., -0.3292, -0.0130,  0.0716]])

In [49]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3970,  0.4024,  1.0612,  ..., -0.0136, -0.3363,  0.6442],
        [-0.5197,  1.0395,  0.2092,  ..., -0.8857, -0.2294,  0.1244],
        [ 0.0057, -0.0707, -0.0804,  ..., -0.3292, -0.0130,  0.0716]])


In [50]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [51]:

criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [52]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [53]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [54]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [55]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [57]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 6s
	Train Loss: 0.259 | Train Acc: 90.09%
	 Val. Loss: 0.325 |  Val. Acc: 87.27%
Epoch: 02 | Epoch Time: 1m 8s
	Train Loss: 0.193 | Train Acc: 92.91%
	 Val. Loss: 0.315 |  Val. Acc: 87.47%
Epoch: 03 | Epoch Time: 1m 8s
	Train Loss: 0.152 | Train Acc: 94.67%
	 Val. Loss: 0.343 |  Val. Acc: 87.58%
Epoch: 04 | Epoch Time: 1m 8s
	Train Loss: 0.116 | Train Acc: 96.09%
	 Val. Loss: 0.316 |  Val. Acc: 87.90%
Epoch: 05 | Epoch Time: 1m 8s
	Train Loss: 0.090 | Train Acc: 97.08%
	 Val. Loss: 0.376 |  Val. Acc: 87.65%


In [58]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.437 | Test Acc: 82.30%
